In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
from scipy.stats import ttest_rel
import warnings
warnings.filterwarnings('ignore')

### Read in the file of metropolitan regions and associated sports teams from [wikipedia_data.html](wikipedia_data.html). Each of these regions may have one or more teams from the "Big 4": NFL (football, in [nfl.csv](nfl.csv)), MLB (baseball, in [mlb.csv](mlb.csv)), NBA (basketball, in [nba.csv](nba.csv) or NHL (hockey, in [nhl.csv](nhl.csv)). 
The metropolitan region file is the "source of authority" for the location of a given sports team. Thus, teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area).
For each sport we are going to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in using 2018 data**. Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. 

In [2]:
nhl_df=pd.read_csv("nhl.csv")
nba_df=pd.read_csv("nba.csv")
mlb_df=pd.read_csv("mlb.csv")
nfl_df=pd.read_csv("nfl.csv")
cities=pd.read_html("wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities.rename(columns={'Population (2016 est.)[8]': 'Population'}, inplace=True)

# NHL 

In [3]:
# Clean up the NHL data and find the win/loss ratio in a new columns

nhl = nhl_df[nhl_df['year'] == 2018]
nhl = nhl.iloc[:, [0, 2,3]]
nhl['W'].replace('[a-zA-Z]+', np.nan, regex=True, inplace=True)
nhl.dropna(inplace = True)
nhl['W'] = nhl['W'].astype(float)
nhl['L'] = nhl['L'].astype(float)
nhl['ratio'] = nhl['W'] / (nhl['W'] + nhl['L'])
nhl['metro'] = ''
for i in nhl.index:
    s = nhl['team'][i] 
    s = s.split()
    nhl['metro'][i] = (s[0]+' '+s[1])
nhl.replace(r'[*]', '', regex=True, inplace=True)
nhl['metro'].replace({'Tampa Bay':'Tampa Bay Area',
                      'Boston Bruins': 'Boston',
                     'Toronto Maple': 'Toronto',
                     'Florida Panthers': 'Miami–Fort Lauderdale',
                     'Detroit Red': 'Detroit',
                     'Montreal Canadiens': 'Montreal',
                     'Ottawa Senators': 'Ottawa',
                     'Buffalo Sabres': 'Buffalo',
                     'Washington Capitals': 'Washington, D.C.',
                     'Pittsburgh Penguins': 'Pittsburgh',
                     'Philadelphia Flyers': 'Philadelphia',
                     'Columbus Blue': 'Columbus',
                     'New Jersey': 'New York City',
                     'Carolina Hurricanes': 'Raleigh',
                     'New York': 'New York City',
                     'Nashville Predators': 'Nashville',
                     'Winnipeg Jets': 'Winnipeg',
                     'Minnesota Wild': 'Minneapolis–Saint Paul',
                     'Colorado Avalanche': 'Denver',
                     'Dallas Stars': 'Dallas–Fort Worth',
                     'Chicago Blackhawks': 'Chicago',
                     'Vegas Golden': 'Las Vegas',
                     'Anaheim Ducks': 'Los Angeles',
                     'Calgary Flames': 'Calgary',
                     'Edmonton Oilers': 'Edmonton',
                     'Vancouver Canucks': 'Vancouver',
                     'Arizona Coyotes': 'Phoenix',
                     'San Jose': 'San Francisco Bay Area'}, inplace=True)


In [4]:
# Clean up the Metropolitan area data for NHL

cities_nhl = cities.iloc[:, [0,1,5]]
cities_nhl['NHL'].replace('[[].*[]]', '', regex=True, inplace=True)
cities_nhl['NHL'].replace('[—]', np.nan, regex=True, inplace=True)
cities_nhl['NHL'].replace('', np.nan, regex=True, inplace=True)
cities_nhl.dropna(inplace=True)
cities_nhl['Population'] = cities_nhl['Population'].astype(np.float)

In [5]:
# Set Metropolitan area as index in both dataframes then match by Metropolitan area
# The final data for NHL has the win/loss ratio and the population of the area

cities_nhl.set_index('Metropolitan area', inplace=True)
nhl.set_index('metro', inplace=True)
    
final = pd.merge(nhl, cities_nhl, left_index=True, right_index=True)
final_nhl = final.groupby(final.index).agg({'ratio': np.mean, 'Population': 'first'})

In [7]:
# Calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL**

stats.pearsonr(final_nhl.Population, final_nhl.ratio)[0]

0.012486162921209923

# NBA

In [8]:
# Clean-up the NBA data and calculate win/loss ratio in a new column

nba = nba_df[nba_df['year'] == 2018]
nba = nba.iloc[:, [0,1,2]]
nba['W'].replace('[a-zA-Z]+', np.nan, regex=True, inplace=True)
nba.dropna(inplace = True)
nba['W'] = nba['W'].astype(np.float)
nba['L'] = nba['L'].astype(np.float)
nba['ratio'] = nba['W'] / (nba['W'] + nba['L'])
nba.replace('[(][\d]{1,2}[)]', '', regex=True, inplace=True)
nba.replace('[*]', '', regex=True, inplace=True)
nba['team'] = nba['team'].str.strip()
nba['metro'] = ''
for i in nba.index:
    s = nba['team'][i] 
    s = s.split()
    nba['metro'][i] = s[0]
nba['metro'].replace({'Indiana': 'Indianapolis',
                     'Miami': 'Miami–Fort Lauderdale',
                     'Washington': 'Washington, D.C.',
                     'New': 'New York City',
                     'Brooklyn': 'New York City',
                     'Golden': 'San Francisco Bay Area',
                     'Oklahoma': 'Oklahoma City',
                     'Utah': 'Salt Lake City',
                     'San': 'San Antonio',
                     'Minnesota': 'Minneapolis–Saint Paul',
                     'Los': 'Los Angeles',
                     'Dallas': 'Dallas–Fort Worth'}, inplace=True)
nba['metro'][20] = 'New Orleans'

In [9]:
# Clean-up the Metropolitan area data for NBA

cities_nba = cities.iloc[:, [0,1,4]]
cities_nba['NBA'].replace('[[].*[]]', '', regex=True, inplace=True)
cities_nba['NBA'].replace('[—]', np.nan, regex=True, inplace=True)
cities_nba['NBA'].replace('', np.nan, regex=True, inplace=True)
cities_nba['NBA'] = cities_nba['NBA'].str.strip()
cities_nba.dropna(inplace=True)
cities_nba['Population'] = cities_nba['Population'].astype(np.float)

In [10]:
# Set Metropolitan area as index in both dataframes then match by Metropolitan area
# The final data for NBA has the win/loss ratio and the population of the area

cities_nba.set_index('Metropolitan area', inplace=True)
nba.set_index('metro', inplace=True)
    
final_nba = pd.merge(nba, cities_nba, left_index=True, right_index=True)
final_nba = final_nba.groupby(final_nba.index).agg({'ratio': np.mean, 'Population': 'first'})

In [11]:
# Calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA**

stats.pearsonr(final_nba.Population, final_nba.ratio)[0]

-0.17657160252844614

# MLB

In [12]:
# Clean-up the MLB data and calculate win/loss ratio in a new column

mlb = mlb_df[mlb_df['year'] == 2018]
mlb = mlb.iloc[:, [0,1,2]]
mlb['W'].replace('[a-zA-Z]+', np.nan, regex=True, inplace=True)
mlb.dropna(inplace = True)
mlb['W'] = mlb['W'].astype(np.float)
mlb['L'] = mlb['L'].astype(np.float)
mlb['ratio'] = mlb['W'] / (mlb['W'] + mlb['L'])
mlb['team'] = mlb['team'].str.strip()
mlb['metro'] = ''
for i in mlb.index:
    s = mlb['team'][i] 
    s = s.split()
    mlb['metro'][i] = s[0]
mlb['metro'].replace({'New': 'New York City',
                     'Tampa': 'Tampa Bay Area',
                     'Minnesota': 'Minneapolis–Saint Paul',
                     'Kansas': 'Kansas City',
                     'Oakland': 'San Francisco Bay Area',
                     'Los': 'Los Angeles',
                     'Texas': 'Dallas–Fort Worth',
                     'Washington': 'Washington, D.C.',
                     'Miami': 'Miami–Fort Lauderdale',
                     'St.': 'St. Louis',
                     'Colorado': 'Denver',
                     'Arizona': 'Phoenix',
                     'San': 'San Francisco Bay Area'}, inplace=True)
mlb['metro'][29] = 'San Diego'

In [13]:
# Clean-up the Metropolitan area data for MLB

cities_mlb = cities.iloc[:, [0,1,3]]
cities_mlb['MLB'].replace('[[].*[]]', '', regex=True, inplace=True)
cities_mlb['MLB'].replace('[—]', np.nan, regex=True, inplace=True)
cities_mlb['MLB'].replace('', np.nan, regex=True, inplace=True)
cities_mlb['MLB'] = cities_mlb['MLB'].str.strip()
cities_mlb.dropna(inplace=True)
cities_mlb['Population'] = cities_mlb['Population'].astype(np.float)

In [14]:
# Set Metropolitan area as index in both dataframes then match by Metropolitan area
# The final data for NBA has the win/loss ratio and the population of the area

cities_mlb.set_index('Metropolitan area', inplace=True)
mlb.set_index('metro', inplace=True)

final_mlb = pd.merge(mlb, cities_mlb, left_index=True, right_index=True)
final_mlb = final_mlb.groupby(final_mlb.index).agg({'ratio': np.mean, 'Population': 'first'})

In [20]:
# Calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB**

stats.pearsonr(final_mlb.Population, final_mlb.ratio)[0]

0.15027698302669307

# NFL

In [21]:
# Clean-up the NFL data and calculate win/loss ratio in a new column

nfl = nfl_df[nfl_df['year'] == 2018]
nfl = nfl.iloc[:, [-4, 1,-2]]
nfl['team'].replace('[*]', '', regex=True, inplace=True)
nfl['team'].replace('[+]', '', regex=True, inplace=True)
nfl['team'] = nfl['team'].str.strip()
nfl['W'].replace('[a-zA-Z]+', np.nan, regex=True, inplace=True)
nfl.dropna(inplace=True)
nfl['W'] = nfl['W'].astype(np.float)
nfl['L'] = nfl['L'].astype(np.float)
nfl['ratio'] = nfl['W'] / (nfl['W'] + nfl['L'])
nfl['metro'] = ''
for i in nfl.index:
    s = nfl['team'][i] 
    s = s.split()
    nfl['metro'][i] = s[0]
nfl['metro'].replace({'New': 'New York City',
                     'Miami': 'Miami–Fort Lauderdale',
                     'Tennessee': 'Nashville',
                     'Kansas': 'Kansas City',
                     'Los': 'Los Angeles',
                     'Oakland': 'San Francisco Bay Area',
                     'Dallas': 'Dallas–Fort Worth',
                     'Washington': 'Washington, D.C.',
                     'Minnesota': 'Minneapolis–Saint Paul',
                     'Green': 'Green Bay',
                     'Carolina': 'Charlotte',
                     'Tampa': 'Tampa Bay Area',
                     'San': 'San Francisco Bay Area',
                     'Arizona': 'Phoenix'}, inplace=True)
nfl['metro'][1] = 'Boston'
nfl['metro'][31] = 'New Orleans'

In [22]:
# Clean-up the Metropolitan area data for NFL

cities_nfl = cities.iloc[:, [0,1,2]]
cities_nfl['NFL'].replace('[[].*[]]', '', regex=True, inplace=True)
cities_nfl['NFL'].replace('[—]', np.nan, regex=True, inplace=True)
cities_nfl['NFL'].replace('', np.nan, regex=True, inplace=True)
cities_nfl.dropna(inplace=True)
cities_nfl['Population'] = cities_nfl['Population'].astype(np.float)

In [23]:
# Set Metropolitan area as index in both dataframes then match by Metropolitan area
# The final data for NBA has the win/loss ratio and the population of the are

cities_nfl.set_index('Metropolitan area', inplace=True)

final_nfl = pd.merge(cities_nfl, nfl, left_index=True, right_on='metro')
final_nfl = final_nfl.groupby('metro').agg({'ratio': np.mean, 'Population': 'first'})

In [24]:
# Calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB**

stats.pearsonr(final_nfl.Population, final_nfl.ratio)[0]

0.004922112149349409

# Let's explore the hypothesis 

Hypothesis: **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. We will explore this with a series of paired **t-tests** between all pairs of sports. Are there any sports where we can reject the null hypothesis? Average values where a sport has multiple teams in one 
region. We will only include, for each sport, cities which have teams engaged in that sport,
drop others as appropriate. 

In [43]:
# Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
# df.loc["NFL","NFL"] should return np.nan

sports = ['NFL', 'NBA', 'NHL', 'MLB']

# create a blanc(with NaN values) 4*4 dataframe 
p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)

NHL = final_nhl.groupby(final_nhl.index).agg({'ratio': np.mean})
NBA = final_nba.groupby(final_nba.index).agg({'ratio': np.mean})
MLB = final_mlb.groupby(final_mlb.index).agg({'ratio': np.mean})
NFL = final_nfl.groupby('metro').agg({'ratio': np.mean})

# a dictionary with team names as keys and their dataframes as values 
f_dic = {'NHL': NHL, 'NBA': NBA, 'MLB': MLB, 'NFL': NFL}

# nested loop to match all possible pairs of teams together and calculate p-values for all possible combinations
for a in sports:
    for b in sports:
        data1 = f_dic[a]
        data2 = f_dic[b]
        
        merge = data1.merge(data2, left_index=True, right_index=True, suffixes=('_'+a, '_'+b))
        test = ttest_rel(merge.values[:,0], merge.values[:,1])[1]
        p_values[a][b] = test

In [44]:
p_values

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030883,0.802069
NBA,0.941792,NaN,0.022297,0.950540
NHL,0.030883,0.022297,NaN,0.000708
MLB,0.802069,0.950540,0.000708,NaN


In [27]:
# for NHL we can reject the null nypothesis with 5% significance level